In [32]:
import numpy as np
import pandas as pd

path = "../files/adult_outcome/salary.csv"
df: pd.DataFrame = pd.read_csv(path)

df = df[["age", "workclass", "education", "race", "sex", "native-country", "salary"]]

df.head(5)

,age,workclass,education,race,sex,native-country,salary
0,39,State-gov,Bachelors,White,Male,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,White,Male,United-States,<=50K
2,38,Private,HS-grad,White,Male,United-States,<=50K
3,53,Private,11th,Black,Male,United-States,<=50K
4,28,Private,Bachelors,Black,Female,Cuba,<=50K


In [33]:
########################################

freq = df["workclass"].value_counts(normalize=True).sort_values(ascending=False)

raras = freq[freq < 0.01].index
df["workclass"] = df["workclass"].where(~df["workclass"].isin(raras), "others")

df["workclass"] = (df["workclass"]
           .astype(str).str.strip().str.lower()
           .replace({"others": "others", "?": "others"})
           .replace({"local-gov": "gov", "state-gov": "gov", "federal-gov": "gov"})
           .replace({"self-emp-not-inc": "self-employed", "self-emp-inc": "self-employed"}))

map_workclass = {'gov': 0, 'self-employed': 1, 'private': 2, 'others': 3}
df['workclass'] = df['workclass'].map(map_workclass)

########################################

map_sex = {' Male': 0, ' Female': 1}
df['sex'] = df['sex'].map(map_sex)

########################################

age_column = df["age"].clip(lower=1, upper=100)

start = ((age_column - 1) // 10) * 10 + 1
bins = list(range(1, 101, 10)) + [101] 
labels = [f"{start}-{start+9}" for start in range(1, 100, 10)]

df["age"] = pd.cut(age_column, bins=bins, right=False, labels=labels, ordered=True)

df["age"] = (df["age"]
           .astype(str).str.strip().str.lower()
           .replace({"1-10": "11-20", "91-100": "71-80", "81-90": "71-80", "71-80": "61-70"}))

map_age = {'11-20': 0, '21-30': 1, '31-40': 2, '41-50': 3, '51-60': 4, '61-70': 5}
df['age'] = df['age'].map(map_age)

df["age"].value_counts()

########################################

df["race"] = (df["race"]
           .astype(str).str.strip().str.lower()
           .replace({"asian-pac-islander": "others", "amer-indian-eskimo": "others", "other": "others"}))

map_race = {'white': 0, 'black': 1, 'others': 2}
df['race'] = df['race'].map(map_race)

########################################

df["native-country"] = (df["native-country"]
           .astype(str).str.strip().str.lower()
           .replace({"mexico": "others", "?": "others", "philippines": "others", "china": "others", "japan": "others", "india": "others", "germany": "others", 
                    "canada": "others", "puerto-rico": "others", "el-salvador": "others", "cuba": "others", "england": "others", "jamaica": "others", "south": "others", 
                    "italy": "others", "dominican-republic": "others", "vietnam": "others", "guatemala": "others", "poland": "others", "columbia": "others", "taiwan": "others", 
                    "haiti": "others", "iran": "others", "holand-netherlands": "others", "scotland": "others", "hungary": "others", "honduras": "others", 
                    "yugoslavia": "others", "laos": "others", "thailand": "others", "cambodia": "others", "trinadad&tobago": "others", "hong": "others", "ireland": "others", "ecuador": "others",
                    "greece": "others", "france": "others", "peru": "others", "nicaragua": "others", "outlying-us(guam-usvi-etc)": "others",}))

map_native_country = {'united-states': 0, 'others': 1}
df['native-country'] = df['native-country'].map(map_native_country)

########################################

df["education"] = (df["education"]
           .astype(str).str.strip().str.lower()
           .replace({"11th": "school", "10th": "school", "7th-8th": "school", "9th": "school", "12th": "school", "1st-4th": "school", "5th-6th": "school"})
           .replace({"assoc-voc": "assoc", "assoc-acdm": "assoc"})
           .replace({"preschool": "school", "prof-school": "school"}))


map_education = {'hs-grad': 0, 'some-college': 1, 'bachelors': 2, 'school': 3, 'assoc': 4, 'masters': 5, 'doctorate': 6}
df['education'] = df['education'].map(map_education)

########################################

map_salary = {' <=50K': 0, ' >50K': 1}
df['salary'] = df['salary'].map(map_salary)

########################################

df = df.astype("float64")

df.head()

,age,workclass,education,race,sex,native-country,salary
0,2.0,0.0,2.0,0.0,0.0,0.0,0.0
1,3.0,1.0,2.0,0.0,0.0,0.0,0.0
2,2.0,2.0,0.0,0.0,0.0,0.0,0.0
3,4.0,2.0,3.0,1.0,0.0,0.0,0.0
4,1.0,2.0,2.0,1.0,1.0,1.0,0.0


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
import seaborn as sns

X = df.drop("salary", axis = 1)
y = df["salary"]

X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size = 0.2, random_state = 42)

print(f"Shape de X_treino :{X_treino.shape}")
print(f"Shape de X_teste: {X_teste.shape}")
print(f"Shape de y_treino: {y_treino.shape}")    
print(f"Shape de y_teste: {y_teste.shape}")

Shape de X_treino :(26048, 6)
Shape de X_teste: (6513, 6)
Shape de y_treino: (26048,)
Shape de y_teste: (6513,)


In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

cat_cols = X.columns.tolist()

preproc = ColumnTransformer(
    transformers=[("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)]
)

pipe = Pipeline([
    ("prep", preproc),
    ("clf", LogisticRegression(max_iter=200, class_weight="balanced"))
])

pipe.fit(X_treino, y_treino)
y_pred = pipe.predict(X_teste)

print("=== Logistic Regression (com OHE) ===")
print("Accuracy:", accuracy_score(y_teste, y_pred))
print("F1-macro:", f1_score(y_teste, y_pred, average="macro"))
print(confusion_matrix(y_teste, y_pred))
print(classification_report(y_teste, y_pred, digits=3))

rf = RandomForestClassifier(
    n_estimators=300,
    random_state=42,
    class_weight="balanced_subsample"
)

rf.fit(X_treino, y_treino)
y_pred_rf = rf.predict(X_teste)

print("\n=== Random Forest (sem OHE) ===")
print("Accuracy:", accuracy_score(y_teste, y_pred_rf))
print("F1-macro:", f1_score(y_teste, y_pred_rf, average="macro"))
print(confusion_matrix(y_teste, y_pred_rf))
print(classification_report(y_teste, y_pred_rf, digits=3))


ModuleNotFoundError: No module named 'xgboost'

In [ ]:
# Cria o padronizador
scaler = StandardScaler()

# Fit e transform somente em treino
X_treino_scaled = scaler.fit_transform(X_treino)

# Transform em teste (e em novos dados)
X_teste_scaled = scaler.transform(X_teste)

# Cria uma figura e um conjunto de subplots.
fig, ax = plt.subplots(1, 2, figsize = (15, 5))

# Primeiro subplot para X_train antes da padronização
sns.boxplot(data = X_treino, ax = ax[0])
ax[0].set_title('X_treino Antes da Padronização')

# Segundo subplot para X_treino após a padronização
sns.boxplot(data = X_treino_scaled, ax = ax[1])
ax[1].set_title('X_treino Depois da Padronização')

plt.show()